In [1]:
from pathlib import Path
from kfp import dsl, compiler, kubernetes, client

/tmp/ipykernel_675/1949069576.py:2: FutureWarning: KFP will drop support for Python 3.8 on Oct 1, 2024. To use new versions of the KFP SDK after that date, you will need to upgrade to Python >= 3.9. See https://devguide.python.org/versions/ for more details.
  from kfp import dsl, compiler, kubernetes, client


In [2]:
kaggle_secret = "kaggle-secret"
root = Path("/")
sa = root/Path("var/run/secrets/kubernetes.io/serviceaccount")
ns = open(sa/"namespace", "r").read()
client = client.Client()

/opt/conda/lib/python3.8/site-packages/kfp/client/client.py:159: FutureWarning: This client only works with Kubeflow Pipeline v2.0.0-beta.2 and later versions.
  warnings.warn(


In [3]:
@dsl.component(packages_to_install=['kaggle==1.6.14'])
def download_data() -> None:
    import os
    import json
    import zipfile
    import subprocess
    
    def init_kaggle():
        # create the Kaggle config directory
        kaggle_config_dir = os.path.join(
            os.path.expandvars('$HOME'), '.kaggle')
        os.makedirs(kaggle_config_dir, exist_ok = True)

        # write the Kaggle config
        api_dict = {
            "username": os.environ['KAGGLE_USERNAME'],
            "key":os.environ['KAGGLE_KEY']}

        with open(os.path.join(kaggle_config_dir, "kaggle.json"), "w", encoding='utf-8') as f:
            json.dump(api_dict, f)

        cmd = f"chmod 600 {kaggle_config_dir}/kaggle.json"
        output = subprocess.check_output(cmd.split(" "))
        
        return output
        
    result = init_kaggle()
    print(result)
    
    import kaggle
    
    isic_data_path = "/data"
    kaggle.api.competition_download_files("isic-2024-challenge", path=isic_data_path)
    
    with zipfile.ZipFile(os.path.join(isic_data_path, "isic-2024-challenge.zip"), 'r') as zip_ref:
        zip_ref.extractall(isic_data_path)
        
    os.listdir(isic_data_path)

/opt/conda/lib/python3.8/site-packages/kfp/dsl/component_decorator.py:119: FutureWarning: The default base_image used by the @dsl.component decorator will switch from 'python:3.8' to 'python:3.9' on Oct 1, 2024. To ensure your existing components work with versions of the KFP SDK released after that date, you should provide an explicit base_image argument and ensure your component works as intended on Python 3.9.
  return component_factory.create_component_from_func(


In [4]:
@dsl.component(packages_to_install=["kubeflow-training==1.8.0"])
def launch_training() -> None:
    from kubeflow.training import TrainingClient, constants
    from kubernetes.client import (V1ObjectMeta,
                                   V1PodTemplateSpec,
                                   V1PodSpec,
                                   V1Volume,
                                   V1PersistentVolumeClaimVolumeSource,
                                   V1EmptyDirVolumeSource,
                                   V1Container,
                                   V1VolumeMount,
                                   V1ResourceRequirements)
    from kubeflow.training.models import (KubeflowOrgV1PyTorchJob,
                                          KubeflowOrgV1PyTorchJobSpec,
                                          KubeflowOrgV1ReplicaSpec,
                                          KubeflowOrgV1RunPolicy)
    
    training_client = TrainingClient(job_kind=constants.PYTORCHJOB_KIND)
    
    # define job's metadata
    pytorch_job_metadata = V1ObjectMeta(name="pytorch-dist-isic-efficientnet")
    pytorch_replica_metadata = V1ObjectMeta(
        annotations={"sidecar.istio.io/inject": "false"})
    
    # define volumes
    data_volume = V1Volume(
        name="isic-data",
        persistent_volume_claim=V1PersistentVolumeClaimVolumeSource(claim_name="isic-data"))
    logs_volume = V1Volume(
        name="isic-logs",
        persistent_volume_claim=V1PersistentVolumeClaimVolumeSource(claim_name="isic-logs"))
    shm_volume = V1Volume(
        name="dshm",
        empty_dir=V1EmptyDirVolumeSource(medium="Memory", size_limit="2Gi"))
    
    # define volume mounts
    data_volume_mount = V1VolumeMount(
        name="isic-data",
        mount_path="/data")
    logs_volume_mount = V1VolumeMount(
        name="isic-logs",
        mount_path="/logs")
    dshm_volume_mount = V1VolumeMount(
        name="dshm",
        mount_path="/dev/shm")
    
    # define job's container
    pytorch_replica_container = V1Container(
        name="pytorch",
        image="dpoulopoulos/pytorch-dist-isic:61a89cd",
        resources=V1ResourceRequirements(
            limits={"nvidia.com/gpu": "1"}),
        volume_mounts=[data_volume_mount, logs_volume_mount, dshm_volume_mount])
    
    # define job's replica spec
    pytorch_replica_template_spec = V1PodSpec(
        volumes=[data_volume, logs_volume, shm_volume],
        containers=[pytorch_replica_container])
    pytorch_replica_template = V1PodTemplateSpec(
        metadata=pytorch_replica_metadata,
        spec=pytorch_replica_template_spec)
    pytorch_replica_spec = KubeflowOrgV1ReplicaSpec(
        replicas=1,
        restart_policy="OnFailure",
        template=pytorch_replica_template)
    pytorch_replica_specs = {
        "Master": pytorch_replica_spec,
        "Worker": pytorch_replica_spec
    }
    
    # define PyTorchJob spec
    pytorch_job_spec = KubeflowOrgV1PyTorchJobSpec(
        pytorch_replica_specs=pytorch_replica_specs,
        run_policy=KubeflowOrgV1RunPolicy())
    
    pytorch_job = KubeflowOrgV1PyTorchJob(
        api_version="kubeflow.org/v1",
        kind="PyTorchJob",
        metadata=pytorch_job_metadata,
        spec=pytorch_job_spec)
    
    training_client.create_job(pytorch_job, namespace="kubeflow-user-example-com")

In [5]:
@dsl.pipeline
def isic_pipeline() -> None:
    # create a PVC to store the dataset
    isic_data_pvc = kubernetes.CreatePVC(
        pvc_name='isic-data',
        access_modes=['ReadWriteMany'],
        size='8.0Gi',
        storage_class_name='longhorn'
    )
    
    # create a PVC to log the training progress
    isic_logs_pvc = kubernetes.CreatePVC(
        pvc_name='isic-logs',
        access_modes=['ReadWriteMany'],
        size='2.0Gi',
        storage_class_name='longhorn'
    )

    download_data_step = download_data().after(isic_data_pvc)
    download_data_step.set_caching_options(enable_caching=False)
    launch_training_step = launch_training().after(download_data_step)
    launch_training_step.set_caching_options(enable_caching=False)

    kubernetes.mount_pvc(
        download_data_step,
        pvc_name=isic_data_pvc.outputs['name'],
        mount_path='/data')
    kubernetes.use_secret_as_env(
        download_data_step,
        secret_name=kaggle_secret,
        secret_key_to_env={'username': 'KAGGLE_USERNAME'})
    kubernetes.use_secret_as_env(
        download_data_step,
        secret_name=kaggle_secret,
        secret_key_to_env={'key': 'KAGGLE_KEY'})

In [6]:
compiler.Compiler().compile(isic_pipeline, package_path='pipeline.yaml')

In [7]:
experiment = client.create_experiment(
    name="isic-experiment",
    description="Skin Cancer Detection with 3D-TBP",
    namespace=ns)

In [8]:
pipeline = client.create_run_from_pipeline_package(
    pipeline_file="pipeline.yaml",
    experiment_name=experiment.display_name,
    namespace=ns,
    run_name="isic-run"
)   